<a href="https://colab.research.google.com/github/chicks2014/sentiment_analysis/blob/main/sentiment_analysis_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.toktok import ToktokTokenizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.head()

,ID,Date,Time,Headline,News,Source,Field,Reaction/Sentiment,Polarity,Subjectivity
0,99248,2002-04-02,00:00:00,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,online_post_1637,Politics,0.330012,0.000000,0.000000
1,10423,2008-09-20,00:00:00,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",new_channel_175,Finance,0.517397,0.100000,0.200000
2,18828,2012-01-28,00:00:00,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",new_channel_175,Finance,0.439257,0.000000,0.041667
3,27788,2015-03-01,00:06:00,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,NaN,Finance,0.449774,-0.166667,0.166667
4,27789,2015-03-01,00:11:00,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,new_channel_1500,Finance,0.512907,0.133333,0.380556


In [ ]:
df_train.shape

(2406, 10)

In [ ]:
df_test.head()

,ID,Date,Time,Headline,News,Source,Field,Polarity,Subjectivity
0,40606,2016-04-07,17:21:21,Draghi: We Face Uncertainty About the Global E...,Hermes Investment Group Chief Economist Neil W...,new_channel_175,Finance,0.000000,0.000000
1,40669,2016-04-07,17:21:23,Meg Whitman explains why Hewlett Packard inves...,Word is that Microsoft tried to buy Mesosphere...,journal_1907,Tech,-0.027778,0.344444
2,40563,2016-04-07,17:22:15,Auditor: Government Will Owe More Money Than E...,An auditor for the Government Accountability O...,journal_1860,Finance,0.100000,0.200000
3,89084,2016-04-07,17:35:07,"The GOP fantasy version of Obama, weighed agai...",The unemployment rate is now down to 5 percent...,online_post_982,Politics,0.047222,0.311111
4,89096,2016-04-07,17:35:08,President Obama has a message for Villanova he...,One of Barack Obama's many duties as president...,online_post_519,Politics,0.500000,0.500000


In [ ]:
df_test.shape

(1022, 9)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2406 entries, 0 to 2405
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  2406 non-null   int64  
 1   Date                2374 non-null   object 
 2   Time                2352 non-null   object 
 3   Headline            2372 non-null   object 
 4   News                2361 non-null   object 
 5   Source              2360 non-null   object 
 6   Field               2377 non-null   object 
 7   Reaction/Sentiment  2406 non-null   float64
 8   Polarity            2406 non-null   float64
 9   Subjectivity        2406 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 188.1+ KB


In [ ]:
df_train.describe()

,ID,Reaction/Sentiment,Polarity,Subjectivity
count,2406.000000,2406.000000,2406.000000,2406.000000
mean,11167.533250,0.507577,0.077957,0.318952
std,21983.954467,0.092792,0.185980,0.252795
min,1.000000,0.137969,-0.800000,0.000000
25%,615.250000,0.444229,0.000000,0.097917
50%,1235.500000,0.512564,0.033333,0.311111
75%,1850.750000,0.573408,0.176015,0.500000
max,99248.000000,0.927028,1.000000,1.000000


# **Data cleaning**

In [ ]:
df_train.Field.unique()

array(['Politics', 'Finance', 'Tech', 'International', nan], dtype=object)

In [ ]:
df_train.isna().sum()

ID                     0
Date                  32
Time                  54
Headline              34
News                  45
Source                46
Field                 29
Reaction/Sentiment     0
Polarity               0
Subjectivity           0
dtype: int64

In [ ]:
df_train.iloc[0]['News']

'Obama Lays Wreath at Arlington National Cemetery. President Barack Obama has laid a wreath at the Tomb of the Unknowns to honor'

In [ ]:
df_train.iloc[0]['Headline']

'Obama Lays Wreath at Arlington National Cemetery'

In [ ]:
!pip install BeautifulSoup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement BeautifulSoup (from versions: 3.2.0, 3.2.1, 3.2.2)
ERROR: No matching distribution found for BeautifulSoup


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def clean_text(text):
  text = re.sub(r'\[[0-9]*\]', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  text = text.lower()
  text = re.sub(r'\d',' ', text)
  text = re.sub(r'\s+',' ', text)
  text = re.sub('\[[^]]*\]', '', text)
  pattern=r'[^a-zA-z0-9\s]'
  text=re.sub(pattern,'',text)
  soup = BeautifulSoup(text, "html.parser")
  text =  soup.get_text()
  return text

In [ ]:
df_train['clean_news'] = df_train.News.apply(lambda x: clean_text(str(x)))

In [ ]:
df_train.head()

,ID,Date,Time,Headline,News,Source,Field,Reaction/Sentiment,Polarity,Subjectivity,clean_news
0,99248,2002-04-02,00:00:00,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,online_post_1637,Politics,0.330012,0.000000,0.000000,obama lays wreath at arlington national cemete...
1,10423,2008-09-20,00:00:00,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",new_channel_175,Finance,0.517397,0.100000,0.200000,tim haywood investment director businessunit h...
2,18828,2012-01-28,00:00:00,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",new_channel_175,Finance,0.439257,0.000000,0.041667,nouriel roubini nyu professor and chairman at ...
3,27788,2015-03-01,00:06:00,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,NaN,Finance,0.449774,-0.166667,0.166667,finlands economy expanded marginally in the th...
4,27789,2015-03-01,00:11:00,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,new_channel_1500,Finance,0.512907,0.133333,0.380556,tourism and public spending continued to boost...


In [ ]:
stopword_list=nltk.corpus.stopwords.words('english')
tokenizer=ToktokTokenizer()

In [ ]:
def simple_stemmer(text):
    lemma = WordNetLemmatizer()
    text= ' '.join([lemma.lemmatize(word) for word in text.split()])
    return text

#Apply function on review column
df_train['clean_news'] = df_train['clean_news'].apply(simple_stemmer)

In [ ]:
for i in range(5):
  print('headline: ')
  print(df_train.iloc[i]['Headline'])
  print(df_train.iloc[i]['News'])
  print(df_train.iloc[i]['clean_news'])
  print('--'*50)

headline: 
Obama Lays Wreath at Arlington National Cemetery
Obama Lays Wreath at Arlington National Cemetery. President Barack Obama has laid a wreath at the Tomb of the Unknowns to honor
obama lay wreath at arlington national cemetery president barack obama ha laid a wreath at the tomb of the unknown to honor
----------------------------------------------------------------------------------------------------
headline: 
A Look at the Health of the Chinese Economy
Tim Haywood, investment director business-unit head for fixed income at Gam, discusses the China beige book and the state of the economy.
tim haywood investment director businessunit head for fixed income at gam discus the china beige book and the state of the economy
----------------------------------------------------------------------------------------------------
headline: 
Nouriel Roubini: Global Economy Not Back to 2008
Nouriel Roubini, NYU professor and chairman at Roubini Global Economics, explains why the global econo

**Stop words**

In [ ]:
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
df_train['clean_news'] = df_train['clean_news'].apply(remove_stopwords)

{'theirs', 'very', "you've", 'to', 'aren', 'being', 'once', 'most', 'himself', 'too', "hadn't", 'haven', 'am', 'shan', 'wasn', 'each', 'between', 'not', "that'll", 'o', 'off', 'ma', 'their', "shouldn't", 'itself', 'about', 'by', 'your', 'under', 'during', 'can', "aren't", 'isn', "won't", 'the', 'has', 'doesn', 'further', 'y', "mustn't", 'into', 'below', 'is', 'yourself', 'them', 'needn', "isn't", 'out', 'above', 'of', 'down', 't', 'had', 'same', 'our', 'or', 'mustn', 'we', "you'll", 'all', 'her', 'because', 'how', 'she', 'while', 'hers', "weren't", 'do', 'hasn', "you'd", 'no', 'so', 'now', "couldn't", 'few', 'a', 'where', 'more', 'that', 'if', 'why', 'having', 'mightn', 'm', "don't", 'at', 'myself', "didn't", 'these', 'should', 'then', 'd', 'you', 'own', 'its', 'his', 'on', 'who', 'but', 'and', "hasn't", "wouldn't", 'me', 'until', 'any', 'ourselves', 'other', 'be', "you're", 'him', 'which', 'have', 'from', 'themselves', "wasn't", 'again', 's', 'with', 'ours', 'don', 'there', 'up', 'in'

In [ ]:
corpus = []
for i in range(0, df_train.shape[0]):
  corpus.append(df_train['clean_news'][i])

In [ ]:
corpus

['obama lay wreath arlington national cemetery president barack obama ha laid wreath tomb unknown honor',
 'tim haywood investment director businessunit head fixed income gam discus china beige book state economy',
 'nouriel roubini nyu professor chairman roubini global economics explains global economy isnt facing condition',
 'finland economy expanded marginally three month ended december contracting previous quarter preliminary figure statistic finland showed monday',
 'tourism public spending continued boost economy january light contraction private consumption export according bank thailand data',
 'attendee expected see latest version microsoft dynamic sl dynamic gp prweb february read full story httpwwwprwebcomreleases prweb htm',
 'ramallah february wafa palestine liberation organization sectretarygeneral saeb erekat thursday expressed concern kenyan president uhuru kenyattas visit jerusalem jordan valley',
 'first lady michelle obama speaks state dinning room white house washi

# **Train model**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
cv = TfidfVectorizer()

In [ ]:
X = cv.fit_transform(corpus).toarray()

In [ ]:
y = df_train['Subjectivity']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=452)

In [ ]:
X_train.shape, y_train.shape

((1684, 7222), (1684,))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor().fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, mean_absolute_error

In [ ]:
mean_absolute_error(y_test, y_pred)

0.16290256132930964

In [ ]:
r2_score(y_test, y_pred)

0.28133881885593903

In [ ]:
mean_squared_error(y_test, y_pred)

0.0505814424173242

In [ ]:
mean_absolute_error(y_test, y_pred)

0.16290256132930964

In [ ]:
score = 100 * max(0, 1 - mean_absolute_error(y_test, y_pred))

In [ ]:
score

83.70974386706904